## Compute Jacobians Matrix for Extended Kalman Filter

---
## Objective

- Easily compute the Jacobian matrix for EKF with SymPy
- SymPy will automatically compute derivatives using symbols


In [12]:
# Import the module and the most relevant functions
import sympy
sympy.init_printing(use_latex='mathjax')
from sympy import symbols, Matrix, latex

---
## Velocity Motion Model

Using the SymPy library implement the Velocity Motion Model that is explained in the lecture slides. For clearness the main equation of the model is reported below.

$$
\begin{pmatrix} x' \\ y' \\ \theta' \end{pmatrix} = g(u_t,x_{t-1}) + \mathcal{N}(0, R_t)= 
\begin{pmatrix} x \\ y \\ \theta \end{pmatrix} + 
\begin{pmatrix} 
- \frac{v_t}{\omega_t }\sin{\theta} + \frac{v_t}{\omega_t }\sin{( \theta + \omega_t \Delta t )} \\
\frac{v_t}{\omega_t }\cos{\theta} - \frac{v_t}{\omega_t }\cos{(\theta + \omega_t \Delta t)} \\
\omega_t \Delta t
\end{pmatrix} + \mathcal{N}(0, R_t)
$$

Then, use the function provided by SymPy to derive the Jacobian matrices w.r.t the state $G_t$ and the input $V_t$ and create a function to evaluate it.

In [13]:
x, y, theta, v, w, dt = symbols('x y theta v w dt')
R = v / w
beta = theta + w * dt
gux = Matrix(
    [
        [x - R * sympy.sin(theta) + R * sympy.sin(beta)],
        [y + R * sympy.cos(theta) - R * sympy.cos(beta)],
        [beta],
    ]
)
gux

⎡  v⋅sin(θ)   v⋅sin(dt⋅w + θ)    ⎤
⎢- ──────── + ─────────────── + x⎥
⎢     w              w           ⎥
⎢                                ⎥
⎢ v⋅cos(θ)   v⋅cos(dt⋅w + θ)     ⎥
⎢ ──────── - ─────────────── + y ⎥
⎢    w              w            ⎥
⎢                                ⎥
⎣            dt⋅w + θ            ⎦

In [14]:
eval_gux = sympy.lambdify((x, y, theta, v, w, dt), gux, 'numpy')

In [15]:
Gt = gux.jacobian(Matrix([x, y, theta]))
eval_Gt = sympy.lambdify((x, y, theta, v, w, dt), Gt, "numpy")
Gt

⎡        v⋅cos(θ)   v⋅cos(dt⋅w + θ)⎤
⎢1  0  - ──────── + ───────────────⎥
⎢           w              w       ⎥
⎢                                  ⎥
⎢        v⋅sin(θ)   v⋅sin(dt⋅w + θ)⎥
⎢0  1  - ──────── + ───────────────⎥
⎢           w              w       ⎥
⎢                                  ⎥
⎣0  0               1              ⎦

In [16]:
Vt = gux.jacobian(Matrix([v, w]))
eval_Vt = sympy.lambdify((x, y, theta, v, w, dt), Vt, "numpy")
Vt

⎡  sin(θ)   sin(dt⋅w + θ)  dt⋅v⋅cos(dt⋅w + θ)   v⋅sin(θ)   v⋅sin(dt⋅w + θ)⎤
⎢- ────── + ─────────────  ────────────────── + ──────── - ───────────────⎥
⎢    w            w                w                2              2      ⎥
⎢                                                  w              w       ⎥
⎢                                                                         ⎥
⎢ cos(θ)   cos(dt⋅w + θ)   dt⋅v⋅sin(dt⋅w + θ)   v⋅cos(θ)   v⋅cos(dt⋅w + θ)⎥
⎢ ────── - ─────────────   ────────────────── - ──────── + ───────────────⎥
⎢   w            w                 w                2              2      ⎥
⎢                                                  w              w       ⎥
⎢                                                                         ⎥
⎣           0                                    dt                       ⎦

---
## Landmarks Measurement Model

As a measurement model we will use **landmarks** described by distance and bearing w.r.t. the robot current pose.

$$
h(x, m) = 
\left(
    \begin{matrix}
    \sqrt{\left(m_x - x\right)^{2} + \left(m_y - y\right)^{2}}\\
    \operatorname{atan2}{\left(m_y - y,m_x - x \right)} - \theta
    \end{matrix}
\right)
$$

Write in the following cell the code to compute the measurement and its Jacobian w.r.t. to the state. Remember to convert the symbolic functions to Python functions that can be evaluated as done in the previous step.

In [17]:
mx, my = symbols("m_x m_y")
hx = Matrix(
    [
        [sympy.sqrt((mx - x) ** 2 + (my - y) ** 2)],
        [sympy.atan2(my - y, mx - x) - theta],
    ]
)
eval_hx = sympy.lambdify((x, y, theta, mx, my), hx, "numpy")

In [18]:
Ht = hx.jacobian(Matrix([x, y, theta]))
eval_Ht = sympy.lambdify((x, y, theta, mx, my), Ht, "numpy")
Ht

⎡          -mₓ + x                      -m_y + y             ⎤
⎢───────────────────────────  ───────────────────────────  0 ⎥
⎢   ________________________     ________________________    ⎥
⎢  ╱         2            2     ╱         2            2     ⎥
⎢╲╱  (mₓ - x)  + (m_y - y)    ╲╱  (mₓ - x)  + (m_y - y)      ⎥
⎢                                                            ⎥
⎢       -(-m_y + y)                   -(mₓ - x)              ⎥
⎢  ──────────────────────       ──────────────────────     -1⎥
⎢          2            2               2            2       ⎥
⎣  (mₓ - x)  + (m_y - y)        (mₓ - x)  + (m_y - y)        ⎦

---

## Odometry Motion Model

Implement the Odometry motion model described by the equations reported below.
1. Get the inputs $\delta_{rot1}$ , $\delta_{trasl}$, $\delta_{rot2}$ from the previous odometry reading $\bar x$ and the current odometry reading $\bar x'$
2. Implement the function $g(u_t,x_{t-1})$ to use this model inside an EKF

$$
\begin{align}
&\delta_{rot1} =& \operatorname{atan_{2}}{\left(\bar{y}'- \bar{y} ,\bar{x}' - \bar{x}  \right)} - \bar{\theta} \\
&\delta_{trasl} =& \sqrt{\left(\bar{x}' - \bar{x} \right)^{2} + \left(\bar{y}' - \bar{y} \right)^{2}} \\
&\delta_{rot2} =& \bar{\theta}' - \delta_{rot1} - \bar{\theta} \\
\end{align}
$$

$$
\begin{pmatrix}
x_t \\ y_t \\ \theta_t
\end{pmatrix} = g(u_t,x_{t-1}) + \mathcal{N}(0, R_t) = 
\begin{pmatrix}
x_{t-1} \\ y_{t-1} \\ \theta_{t-1}
\end{pmatrix} + 
\left(
    \begin{matrix}
    \delta_{trasl} \cos{\left(\delta_{rot1} + \theta \right)} \\
    \delta_{trasl} \sin{\left(\delta_{rot1} + \theta \right)} \\
    \delta_{rot1} + \delta_{rot2}
    \end{matrix} 
\right) + \mathcal{N}(0, R_t)
$$

In [19]:
x, y, theta = symbols(r"x y \theta")
rot1, trasl, rot2 = symbols(r"\delta_{rot1} \delta_{trasl} \delta_{rot2}")
gux_odom = Matrix([
    [x + trasl * sympy.cos(theta + rot1)],
    [y + trasl * sympy.sin(theta + rot1)],
    [theta + rot1 + rot2],
])
Gt_odom = gux_odom.jacobian(Matrix([x, y, theta]))
Vt_odom = gux_odom.jacobian(Matrix([rot1, trasl, rot2]))

args = (x, y, theta, rot1, trasl, rot2)
eval_Gt_odom = sympy.lambdify(args, Gt_odom, "numpy")
eval_Vt_odom= sympy.lambdify(args, Vt_odom, "numpy")

In [20]:
Gt

⎡        v⋅cos(θ)   v⋅cos(dt⋅w + θ)⎤
⎢1  0  - ──────── + ───────────────⎥
⎢           w              w       ⎥
⎢                                  ⎥
⎢        v⋅sin(θ)   v⋅sin(dt⋅w + θ)⎥
⎢0  1  - ──────── + ───────────────⎥
⎢           w              w       ⎥
⎢                                  ⎥
⎣0  0               1              ⎦

In [21]:
Vt

⎡  sin(θ)   sin(dt⋅w + θ)  dt⋅v⋅cos(dt⋅w + θ)   v⋅sin(θ)   v⋅sin(dt⋅w + θ)⎤
⎢- ────── + ─────────────  ────────────────── + ──────── - ───────────────⎥
⎢    w            w                w                2              2      ⎥
⎢                                                  w              w       ⎥
⎢                                                                         ⎥
⎢ cos(θ)   cos(dt⋅w + θ)   dt⋅v⋅sin(dt⋅w + θ)   v⋅cos(θ)   v⋅cos(dt⋅w + θ)⎥
⎢ ────── - ─────────────   ────────────────── - ──────── + ───────────────⎥
⎢   w            w                 w                2              2      ⎥
⎢                                                  w              w       ⎥
⎢                                                                         ⎥
⎣           0                                    dt                       ⎦